SPAM or HAM Classification Problem

In [ ]:
#Load Libraies

In [100]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

%matplotlib inline

In [101]:
import nltk
import string
from nltk import word_tokenize
from nltk.util import bigrams, trigrams
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

In [102]:
#Access File

In [103]:
df= pd.read_csv('C:\\Users\\edingha\\Documents\\Business Analytics\\TextAnalytics\\Open_Exam_Data_set\\spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [104]:
#Removing Unnamed Columns
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df = df.rename(columns={"v1":"Class", "v2":"Message"})

In [105]:
df.head()
df.tail()

,Class,Message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [106]:
#Preprocessing

In [107]:
NLTK_STOPWORDS = set(stopwords.words('english'))

#Removing digits in text
def remove_numbers_in_string(s):
      
    return s.translate(string.digits)
#Lower case Conversion
def lowercase_remove_punctuation(s):
    print (s)
    s = s.lower()
    s = s.translate(string.punctuation)
    return s

#Remove StopWords  
def remove_stopwords(s):
    token_list = nltk.word_tokenize(s)
    exclude_stopwords = lambda token : token not in NLTK_STOPWORDS
    return ' '.join(filter(exclude_stopwords, token_list))

#Stemming

def stem_token_list(token_list):
    STEMMER = PorterStemmer()
    return [STEMMER.stem(tok) for tok in token_list]

def restring_tokens(token_list):
    return ' '.join(token_list)

def preprocessing(s):
    s = remove_numbers_in_string(s)
    s = lowercase_remove_punctuation(s)
    s = remove_stopwords(s)
    token_list = nltk.word_tokenize(s)
    token_list = stem_token_list(token_list)
    return restring_tokens(token_list)

In [108]:
initial_features = ['Class', 'Message']
df_p = df[initial_features]
df_p['Message'] = df_p['Message'].apply(preprocessing)

for idx in range(5):
    print (df_p.Message[idx])
    print ()

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
U dun say so early hor... U c already then say...
Nah I don't think he goes to usf, he lives around here though
FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
Even my brother is not like to speak with me. They treat me like aids patent.
As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Had your mobile 11 months or more? U R enti

In [109]:
#Convert it to Lowercase

In [110]:
#Remove stop words

In [111]:
#Normalization
def normalize(s):
    for p in string.punctuation:
        s = s.replace(p, '')
 
    return s.lower().strip()

In [112]:

#Use sentence tokenizer from NLTK
from nltk.tokenize import sent_tokenize
for idx in range(5572):
    text=df_p.Message[idx]
    text=normalize(text)
    sent_tokenize_list = sent_tokenize(text)
    
    len(sent_tokenize_list)
    #print(sent_tokenize_list)
    #print ()

In [113]:
#Load sklearn Libraries
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import *
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, roc_auc_score

In [114]:
#Modelling
TEST_SIZE = 0.40
train_X, test_X, train_y, test_y = train_test_split(df_p.Message,df_p.Class,test_size=TEST_SIZE,random_state=42)
    

In [115]:
#Convert: Converting Raw text into numerical fetures (BOW, TF-IDF and W2V)
#http://datameetsmedia.com/bag-of-words-tf-idf-explained/
bag_of_words_vectorizer = CountVectorizer(analyzer = "word",
                                          tokenizer = None,    
                                          preprocessor = None,
                                          ngram_range = (1, 1),
                                          binary = False,
                                          strip_accents='unicode')

In [116]:
bow_feature_matrix_train = bag_of_words_vectorizer.fit_transform(train_X)
bow_feature_matrix_test = bag_of_words_vectorizer.transform(test_X)
bow_feature_matrix_train, bow_feature_matrix_test

(<3343x5787 sparse matrix of type '<class 'numpy.int64'>'
 	with 28816 stored elements in Compressed Sparse Row format>,
 <2229x5787 sparse matrix of type '<class 'numpy.int64'>'
 	with 17249 stored elements in Compressed Sparse Row format>)

In [117]:
#Multinominla NaiveBayes
multinomial_nb_classifier = MultinomialNB()
multinomial_nb_classifier.fit(bow_feature_matrix_train, train_y)
multinomial_nb_prediction = multinomial_nb_classifier.predict(bow_feature_matrix_test)

In [118]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix,classification_report

In [119]:
print(confusion_matrix(test_y,multinomial_nb_prediction))
print('\n')
print(classification_report(test_y,multinomial_nb_prediction))

[[1918   12]
 [  28  271]]


             precision    recall  f1-score   support

        ham       0.99      0.99      0.99      1930
       spam       0.96      0.91      0.93       299

avg / total       0.98      0.98      0.98      2229



In [120]:
##Tf-IDF
from sklearn.feature_extraction.text import  TfidfTransformer

In [121]:
from sklearn.pipeline import Pipeline

In [122]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [123]:
X=df_p.Message
y=df_p.Class
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4,random_state=42)

In [124]:
# May take some time
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [125]:
predictions = pipeline.predict(X_test)

In [126]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[1930    0]
 [  76  223]]
             precision    recall  f1-score   support

        ham       0.96      1.00      0.98      1930
       spam       1.00      0.75      0.85       299

avg / total       0.97      0.97      0.96      2229



In [127]:
#########  Lexicon Model

In [128]:
lex_file = open("C:\\Users\\edingha\\Documents\\Business Analytics\\TextAnalytics\\AFINN-111.csv")

In [129]:
lexicons = {}
records = lex_file.readlines()
for record in records:
    #print(record) - line contains newline charecter
    #print(record.rstrip('\n').split(",")) - to remove new line charecter
    lexicons[record.rstrip('\n').split(",")[0]] = int(record.rstrip('\n').split(",")[1])
#print(lexicons)
#lexicons["abandon"]

In [135]:
#Strip the records and create a word list for each tweet
word_list = []
for idx in range(5572):
    text=df_p.Message[idx]
    text=normalize(text)
    words = []
    token_list = nltk.word_tokenize(text)
    #print(tokens)
    for token in token_list:
        words.append(token.lower())
    word_list.append(words)  
    


In [136]:
word_list

[['go',
  'jurong',
  'point',
  'crazy',
  'avail',
  'bugi',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'cine',
  'got',
  'amor',
  'wat'],
 ['ok', 'lar', 'joke', 'wif', 'u', 'oni'],
 ['free',
  'entri',
  '2',
  'wkli',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkt',
  '21st',
  'may',
  '2005',
  'text',
  'fa',
  '87121',
  'receiv',
  'entri',
  'question',
  'std',
  'txt',
  'rate',
  'c',
  's',
  'appli',
  '08452810075over18',
  's'],
 ['u', 'dun', 'say', 'earli', 'hor', 'u', 'c', 'alreadi', 'say'],
 ['nah', 'nt', 'think', 'goe', 'usf', 'live', 'around', 'though'],
 ['freemsg',
  'hey',
  'darl',
  's',
  '3',
  'week',
  's',
  'word',
  'back',
  'd',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chg',
  'send',
  'å£150',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'mell',
  'mell',
  'oru',
  'minnaminungint',
  'nurungu',
  'vettam',
  'set',
  'callertun',
  'cal

In [137]:
strength = []
for tweet in word_list:
    score = 0
    for word in tweet:
        if word in (lexicons):
            score = score + lexicons[word]
    strength.append(score)


In [138]:
strength

[1,
 2,
 5,
 0,
 0,
 6,
 4,
 0,
 6,
 2,
 1,
 4,
 0,
 7,
 0,
 0,
 0,
 0,
 2,
 -2,
 0,
 6,
 -1,
 0,
 0,
 -2,
 3,
 3,
 0,
 5,
 2,
 3,
 0,
 -2,
 2,
 0,
 0,
 0,
 0,
 0,
 4,
 5,
 2,
 0,
 7,
 0,
 0,
 2,
 3,
 -5,
 0,
 4,
 -2,
 9,
 0,
 0,
 -1,
 0,
 1,
 0,
 -5,
 11,
 0,
 0,
 1,
 3,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 -2,
 0,
 1,
 1,
 -2,
 13,
 0,
 0,
 1,
 0,
 -1,
 0,
 0,
 1,
 0,
 0,
 4,
 5,
 5,
 0,
 0,
 6,
 0,
 0,
 0,
 2,
 0,
 5,
 0,
 4,
 0,
 -1,
 0,
 2,
 0,
 3,
 0,
 0,
 3,
 -2,
 0,
 2,
 0,
 0,
 2,
 0,
 6,
 3,
 0,
 0,
 0,
 1,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 6,
 0,
 0,
 4,
 0,
 4,
 -1,
 0,
 3,
 -3,
 0,
 0,
 -1,
 2,
 3,
 0,
 0,
 1,
 0,
 -6,
 5,
 0,
 0,
 3,
 0,
 3,
 0,
 4,
 0,
 0,
 0,
 -2,
 3,
 0,
 0,
 1,
 0,
 10,
 0,
 -1,
 2,
 5,
 0,
 -3,
 2,
 0,
 0,
 3,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 7,
 0,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 3,
 0,
 -1,
 0,
 3,
 0,
 0,
 2,
 0,
 0,
 5,
 -1,
 2,
 -2,
 0,
 1,
 0,
 -1,
 1,
 4,
 0,
 0,
 0,
 2,

In [139]:
for idx in range(5572):
   
    
    print(df_p.Message[idx], df_p.Class[idx], strength[idx])

go jurong point , crazy.. avail bugi n great world la e buffet ... cine got amor wat ... ham 1
ok lar ... joke wif u oni ... ham 2
free entri 2 wkli comp win fa cup final tkt 21st may 2005. text fa 87121 receiv entri question ( std txt rate ) & c 's appli 08452810075over18 's spam 5
u dun say earli hor ... u c alreadi say ... ham 0
nah n't think goe usf , live around though ham 0
freemsg hey darl 's 3 week 's word back ! 'd like fun still ? tb ok ! xxx std chg send , å£1.50 rcv spam 6
even brother like speak . treat like aid patent . ham 4
per request 'mell mell ( oru minnaminungint nurungu vettam ) ' set callertun caller . press *9 copi friend callertun ham 0
winner ! ! valu network custom select receivea å£900 prize reward ! claim call 09061701461. claim code kl341 . valid 12 hour . spam 6
mobil 11 month ? u r entitl updat latest colour mobil camera free ! call mobil updat co free 08002986030 spam 2
'm gon na home soon n't want talk stuff anymor tonight , k ? 've cri enough today . h

In [140]:
strength=np.array(strength)

In [141]:
strength

array([ 1,  2,  5, ...,  0, -1,  4])

In [142]:
df_strength=pd.DataFrame(strength)

In [150]:
df_p['Strength']=df_strength[0]

In [151]:
df_p

,Class,Message,Strength
0,ham,"go jurong point , crazy.. avail bugi n great w...",1
1,ham,ok lar ... joke wif u oni ...,2
2,spam,free entri 2 wkli comp win fa cup final tkt 21...,5
3,ham,u dun say earli hor ... u c alreadi say ...,0
4,ham,"nah n't think goe usf , live around though",0
5,spam,freemsg hey darl 's 3 week 's word back ! 'd l...,6
6,ham,even brother like speak . treat like aid patent .,4
7,ham,per request 'mell mell ( oru minnaminungint nu...,0
8,spam,winner ! ! valu network custom select receivea...,6
9,spam,mobil 11 month ? u r entitl updat latest colou...,2


In [169]:
df_p['Predicted_Class']=df_p['Strength'].apply(lambda x:"ham" if x>=0 else "spam")

In [170]:
df_p

,Class,Message,Strength,Predicted_Class
0,ham,"go jurong point , crazy.. avail bugi n great w...",1,ham
1,ham,ok lar ... joke wif u oni ...,2,ham
2,spam,free entri 2 wkli comp win fa cup final tkt 21...,5,ham
3,ham,u dun say earli hor ... u c alreadi say ...,0,ham
4,ham,"nah n't think goe usf , live around though",0,ham
5,spam,freemsg hey darl 's 3 week 's word back ! 'd l...,6,ham
6,ham,even brother like speak . treat like aid patent .,4,ham
7,ham,per request 'mell mell ( oru minnaminungint nu...,0,ham
8,spam,winner ! ! valu network custom select receivea...,6,ham
9,spam,mobil 11 month ? u r entitl updat latest colou...,2,ham


In [171]:
total=0
for idx in range(5572):
    
    if df_p.Class[idx]==df_p.Predicted_Class[idx]:
        total=total+1
        #print(total)
        
accuracy=total/5572
accuracy
        

0.7823043790380474

In [172]:
total

4359